In [ ]:
import tkinter as tk
from tkinter import messagebox, scrolledtext
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as sm
import time
import gc
data = {
    'Feature1': [1, 2, 3, 4, 5],
    'Feature2': [5, 4, 3, 2, 1],
    'Feature3': [1, 2, 2, 3, 3],
    'Feature4': [4, 5, 5, 4, 4],
    'Feature5': [0, 1, 1, 0, 0],
    'Feature6': [3, 3, 4, 4, 5],
    'opt': [0, 1, 1, 0, 1]  # Target variable
}

dt = pd.DataFrame(data)
# Initialize global variables
stp = [[0, 0, 0, 0, 0, 0]]
columns_ = ['Col1', 'Col2', 'Col3', 'Col4', 'Col5', 'Col6']
contr = pd.DataFrame(stp, columns=columns_)
ent_ = pd.DataFrame([[2,3,4,5,6,7]], columns=columns_)  # Dummy DataFrame for 'ent_'
dt = pd.DataFrame(data)  # Placeholder for your actual DataFrame


def update_stp():
    global stp, contr
    try:
        stp_values = [float(entry.get()) for entry in entries]
        stp = [stp_values]
        contr = pd.DataFrame(stp, columns=columns_)
        messagebox.showinfo("Success", "Values updated successfully!")
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numbers.")

def run_loop():
    global contr, dt, ent_

    condition_met = False
    start_time = time.time()
    time_limit = 10
    iteration = 0

    while not condition_met:
        try:
            # Track elapsed time
            elapsed_time = time.time() - start_time
            if elapsed_time > time_limit:
                result_text.insert(tk.END, "Time limit exceeded. Stopping the loop.\n")
                break

            iteration += 1
            result_text.insert(tk.END, f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds\n")

            # Reset index for contr DataFrame
            contr.reset_index(drop=True, inplace=True)

            # Concatenate and process DataFrames
            tt2 = pd.concat([ent_, contr], axis=1)
            tt2.fillna(0, inplace=True)
            tt2 = pd.DataFrame(tt2.sum()).T

            tt2_test = tt2[[col for col in columns_ if col in tt2.columns]]

            x_ = dt[[]]
            y_ = dt['opt']

            x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9)
            model = LogisticRegression(penalty='l2', solver='liblinear', class_weight='balanced')
            model.fit(x_, y_)
            y_pred2 = model.predict(tt2)
            y_pred_prob2 = model.predict_proba(tt2)[:, 1]  # Get the probability of the positive class
            y_pred_score2 = model.score(x_test_3, y_test_3)

            # Inferência causal
            model = sm.Logit(y_train_3, x_train_3)
            co = model.fit(method='bfgs', cov_type='HC3').params
            coe = pd.DataFrame(co)
            coe.columns = ['coeff']
            exp_coeff = np.exp(coe) / (np.exp(coe) + 1)
            exp_coeff = pd.DataFrame(exp_coeff)
            exp_coeff.columns = ['exp_coeff']
            results = model.fit(method='bfgs', cov_type='HC3')
            results_summary = results.summary()
            results_as_html = results_summary.tables[1].as_html()
            df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
            df_results['exp_coeff'] = exp_coeff
            impact_var2 = df_results.loc[df_results['P>|z|'] < 0.05]

            results_list = []
            for i in impact_var2.index:
                value = impact_var2.loc[i, 'exp_coeff']

                if i in contr.columns:
                    correction = tt2_test[i].median()

                    if value < 0.5:
                        result = (value - 0.5) * correction
                    elif value > 0.5:
                        result = (value - 0.5) * correction
                    else:
                        result = 0

                    results_list.append({'Column': i, 'Alteração': result})

            results_df_ = pd.DataFrame(results_list)
            dd = results_df_.T
            new_header = dd.iloc[0]
            dd = dd[1:]
            dd.columns = new_header
            dd.reset_index(drop=True, inplace=True)

            common_columns = list(set(contr.columns) & set(dd.columns))
            contr_common = contr[common_columns]
            dd_common = dd[common_columns]

            combined_df = pd.concat([contr_common, dd_common], ignore_index=True)
            final = combined_df.sum().reset_index()
            tt = final.T
            new_header = tt.iloc[0]
            tt = tt[1:]
            tt.columns = new_header
            tt.reset_index(drop=True, inplace=True)

            contr.reset_index(drop=True, inplace=True)
            tt.reset_index(drop=True, inplace=True)

            common_columns = contr.columns.intersection(tt.columns)
            contr.update(tt[common_columns])
            contr.reset_index(inplace=True)
            contr.drop(columns=['index'], inplace=True)

            gc.collect()

            # Check if the condition is met
            if np.all(y_pred_prob2 >= 0.99) and np.all(y_pred2 == 1):
                condition_met = True

        except Exception as e:
            result_text.insert(tk.END, f"An error occurred: {e}\n")
            continue

    # After the loop ends, display the results
    result_text.insert(tk.END, "Final Results:\n")
    result_text.insert(tk.END, "Model Performance:\n")
    result_text.insert(tk.END, f"Accuracy Score: {y_pred_score2:.4f}\n")
    result_text.insert(tk.END, "\nPredicted Classes:\n")
    result_text.insert(tk.END, pd.DataFrame({'Predicted Class': y_pred2}).to_string() + "\n")
    result_text.insert(tk.END, "\nPredicted Probabilities:\n")
    result_text.insert(tk.END, pd.DataFrame({'Predicted Probability': y_pred_prob2}).to_string() + "\n")
    result_text.insert(tk.END, "\nResults DataFrame:\n")
    result_text.insert(tk.END, results_df_.to_string() + "\n")
    result_text.insert(tk.END, "\nUpdated 'contr' DataFrame:\n")
    result_text.insert(tk.END, contr.to_string() + "\n")

# Set up the Tkinter window
root = tk.Tk()
root.title("Processing Loop")

# Set up the input section for `stp`
tk.Label(root, text="Enter values for stp:").pack()
entries = []
for col in columns_:
    tk.Label(root, text=col).pack()
    entry = tk.Entry(root)
    entry.pack()
    entries.append(entry)

update_button = tk.Button(root, text="Update stp", command=update_stp)
update_button.pack()

run_button = tk.Button(root, text="Run Processing Loop", command=run_loop)
run_button.pack()
result_text = scrolledtext.ScrolledText(root, width=100, height=30)
result_text.pack()

root.mainloop()